In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("../input/english-handwritten-characters-dataset/english.csv")

In [ ]:
df.columns

In [ ]:
X = df["image"]
Y = df["label"]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3)

In [ ]:
X_train.shape,X_test.shape

In [ ]:
import os 
os.mkdir("train")
os.mkdir("test")

In [ ]:
for image in df.values:
    if not os.path.isdir("train/"+image[1]):
        os.mkdir("train/"+image[1])
    if not os.path.isdir("test/"+image[1]):
        os.mkdir("test/"+image[1])

In [ ]:
train_df = pd.DataFrame([X_train,Y_train]).T
test_df = pd.DataFrame([X_test,Y_test]).T

In [ ]:
import glob
import shutil
for image in train_df.values:
    img_pth = "../input/english-handwritten-characters-dataset/" + image[0]
    shutil.copy(img_pth,"train/"+image[1])

In [ ]:
for image in test_df.values:
    img_pth = "../input/english-handwritten-characters-dataset/" + image[0]
    shutil.copy(img_pth,"test/"+image[1])

In [ ]:
!ls train/0

In [ ]:
import cv2
import matplotlib.pyplot as plt
img = cv2.imread("train/0/img001-002.png")
plt.imshow(img)
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [ ]:
def build_model(n_inputs,n_outputs):
    model = ResNet50(input_shape=[n_inputs,n_inputs] + [3], weights='imagenet', include_top=False)
    #model.add(tf.keras.layers.InputLayer(input_shape=(n_inputs,n_inputs,1)))
    conv_1 = tf.keras.layers.Conv2D(50,(3,3),activation="relu")(model.output)
    pool_1 = tf.keras.layers.MaxPool2D((2,2))(conv_1)
#     conv_2 = tf.keras.layers.Conv2D(60,(3,3),activation="relu",padding="same")(pool_1)
#     pool_2 = tf.keras.layers.MaxPool2D((2,2))(conv_2)
#     conv_3 = tf.keras.layers.Conv2D(30,(3,3),activation="relu")(pool_2)
#     pool_3 = tf.keras.layers.MaxPool2D((2,2))(conv_3)
#     conv_4 = tf.keras.layers.Conv2D(10,(3,3),activation="relu")(pool_3)
#     pool_4 = tf.keras.layers.MaxPool2D((2,2))(conv_4)
    fcn_1 = tf.keras.layers.Flatten()(pool_1)
    fcn_2 = tf.keras.layers.Dense(240,activation="relu")(fcn_1)
    fcn_3 = tf.keras.layers.Dense(120,activation="relu")(fcn_2)
    output = tf.keras.layers.Dense(n_outputs,activation="softmax")(fcn_3)
    return tf.keras.models.Model(model.input,output)

In [ ]:
n_inputs = 128
n_outputs = df["label"].unique().size
model = build_model(n_inputs,n_outputs)

In [ ]:
#model.summary()

In [ ]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory("train/",
                                                 target_size = (n_inputs,n_inputs),
                                                 batch_size = 20,
                                                 class_mode = 'categorical',shuffle=True)
test_set = test_datagen.flow_from_directory("test/",
                                            target_size = (n_inputs,n_inputs),
                                            batch_size = 20,
                                            class_mode = 'categorical',shuffle=True)

In [ ]:
from tensorflow.keras.optimizers import Adam
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer=Adam(learning_rate=1e-3),
  metrics=["accuracy"]
)

In [ ]:
model.fit_generator(training_set,validation_data = test_set, epochs=50,steps_per_epoch=120,validation_steps=51)

In [ ]:
model.save("model.h5")

In [ ]:
len(test_df.values)
test_df.values[0][0]

In [ ]:
import cv2 as cv
# Print the class indices 
print("Prediction Dict: ", training_set.class_indices)
# Predict on the test data 
pred = model.predict(test_set)
# Create a class/labels dictionary
classDict = {
            0: "0", 1: "1", 2: "2", 3: "3", 4: "4", 5: "5", 6: "6", 7: "7", 8: "8", 9: "9", 10: "A",
            11: "B", 12: "C", 13: "D", 14: "E", 15: "F", 16: "G", 17: "H", 18: "I", 19: "J", 20: "K",
            21: "L", 22: "M", 23: "N", 24: "O", 25: "P", 26: "Q", 27: "R", 28: "S", 29: "T", 30: "U",
            31: "V", 32: "W", 33: "X", 34: "Y", 35: "Z", 36: "a", 37: "b", 38: "c", 39: "d", 40: "e",
            41: "f", 42: "g", 43: "h", 44: "i", 45: "j", 46: "k", 47: "l", 48: "m", 49: "n", 50: "o",
            51: "p", 52: "q", 53: "r", 54: "s", 55: "t", 56: "u", 57: "v", 58: "w", 59: "x", 60: "y",
            61: "z"}

# Make a data frame that contains the probability for each class
outputDf = pd.DataFrame(pred)
# Get the index of the max probability from the output Data frame
maxIndex = list(outputDf.idxmax(axis=1))
# Print the max index
print("Max index: ", maxIndex)
# Make a loop in range the length of the test data (5)
DATADIR = "../input/english-handwritten-characters-dataset"
for i in range(len(test_df.values)):
    # Read the image 
    print(DATADIR + '/' + test_df.values[i][0])
    image = cv.imread(DATADIR + '/' + test_df.values[i][0])
    # The title of the plot which is the predicted label
    plt.title(classDict.get(maxIndex[i], "error"))
    # Show the actual image
    if i==10:
        break
    plt.imshow(image)
    plt.show()